# Segmenting and Clustering Neighborhoods in Toronto

In [2]:
#import necessary libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # library for import html-file
from requests.packages.urllib3.exceptions import InsecureRequestWarning 
#requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
#from requests_html import HTMLSession

import csv
from bs4 import BeautifulSoup
import re
import sys
#import configurate as conf #load module ../configurate.py


In [29]:
#загрузка страницы и выделение таблицы из общего текста
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
responseText = r.text
soup = BeautifulSoup(responseText, "html.parser")

resultTable = soup.table.tbody
#print(resultTable.prettify())

In [89]:
# Handle column names
headline = resultTable.findAll('th')
#print(headline)
n_col = 0
column_names = []
for col_ind in headline:
    column_names.append(col_ind.get_text().strip())
    n_col = n_col + 1
df=pd.DataFrame(columns = column_names)

In [90]:
# Handle row contents
Row_All = resultTable.findAll('tr')
n_row = 0
row_df={}
for Row in Row_All:
    columns = Row.findAll('td')
    line = []
    i = 0
    for col in columns:
        if(len(col)>0): 
            line.append((col.text).strip())
            row_df[column_names[i]] = line[i]
            i = i+1
    if(i>0):
        df = df.append(row_df, ignore_index=True)
        n_row = n_row + 1
print (n_row)
print(df.head())
print(df.tail())

288
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
    Postcode       Borough          Neighbourhood
283      M8Z     Etobicoke              Mimico NW
284      M8Z     Etobicoke     The Queensway West
285      M8Z     Etobicoke  Royal York South West
286      M8Z     Etobicoke         South of Bloor
287      M9Z  Not assigned           Not assigned


In [91]:
#Delete rows where [Borough]=N/A
df.drop(df[(df.Borough == 'Not assigned')].index, inplace=True)
df.reset_index(inplace=True)
del df['index']
print(df.head())
print(df.tail())

  Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M5A  Downtown Toronto       Regent Park
4      M6A        North York  Lawrence Heights
    Postcode    Borough             Neighbourhood
206      M8Z  Etobicoke  Kingsway Park South West
207      M8Z  Etobicoke                 Mimico NW
208      M8Z  Etobicoke        The Queensway West
209      M8Z  Etobicoke     Royal York South West
210      M8Z  Etobicoke            South of Bloor


In [92]:
#Merge Borough
df=df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
print(df.head())
print(df.tail())

  Postcode      Borough                         Neighbourhood
0      M1B  Scarborough                         Rouge,Malvern
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Guildwood,Morningside,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae
    Postcode    Borough                                      Neighbourhood
98       M9N       York                                             Weston
99       M9P  Etobicoke                                          Westmount
100      M9R  Etobicoke  Kingsview Village,Martin Grove Gardens,Richvie...
101      M9V  Etobicoke  Albion Gardens,Beaumond Heights,Humbergate,Jam...
102      M9W  Etobicoke                                          Northwest


In [93]:
#Replace empty Neighbourhood
n_row=df.shape[0]
for i in range(0,n_row):
    if (df.loc[i,'Neighbourhood']=='Not assigned'):
        print (i)
        df.loc[i,'Neighbourhood']=df.loc[i,'Borough']
        print (df.loc[i,'Neighbourhood'])

85
Queen's Park


In [94]:
df.shape

(103, 3)